# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Li  ->  J. Li  |  ['J. Li']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
S. Li  ->  S. Li  |  ['S. Li']
J. Li  ->  J. Li  |  ['J. Li']
Arxiv has 59 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2311.01721


extracting tarball to tmp_2311.01721... done.


list index out of range
Retrieving document from  https://arxiv.org/e-print/2311.01761


extracting tarball to tmp_2311.01761...

 done.
Retrieving document from  https://arxiv.org/e-print/2311.01982


extracting tarball to tmp_2311.01982...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2311.01721-b31b1b.svg)](https://arxiv.org/abs/arXiv:2311.01721) | **Tentative detection of cyanoformamide NCCONH2 in space**  |
|| <mark>J. Li</mark>, et al. -- incl., <mark>X. Zhang</mark>, <mark>S. Li</mark> |
|*Appeared on*| *2023-11-06*|
|*Comments*| *20 pages, 6 figures, 2 tables, accepted by PASJ. arXiv admin note: text overlap with arXiv:2108.05001*|
|**Abstract**| The peptide-like molecules, cyanoformamide (NCCONH2), is the cyano (CN) derivative of formamide (NH2CHO). It is known to play a role in the synthesis of nucleic acid precursors under prebiotic conditions. In this paper, we present a tentative detection of NCCONH2 in the interstellar medium (ISM) with the Atacama Large Millimeter/submillimeter Array (ALMA) archive data. Ten unblended lines of NCCONH2 were seen around 3sigma noise levels toward Sagittarius B2(N1E), a position that is slightly offset from the continuum peak. The column density of NCCONH2 was estimated to be 2.4\times 10^15 cm ^-2, and the fractional abundance of NCCONH2 toward Sgr B2(N1E) was 6.9\times10^-10. The abundance ratio between NCCONH2 and NH2CHO is estimated to be ~0.01. We also searched for other peptide-like molecules toward Sgr B2(N1E). The abundances of NH2CHO, CH3NCO and CH3NHCHO toward Sgr B2(N1E) were about one tenth of those toward Sgr B2(N1S), while the abundances of CH3CONH2 was only one twentieth of that toward Sgr B2(N1S). |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2311.01761-b31b1b.svg)](https://arxiv.org/abs/arXiv:2311.01761) | **ProPane: Image Warping with Fire**  |
|| A. S. G. Robotham, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2023-11-06*|
|*Comments*| *16 pages, 13 figures, 5 tables, submitted to MNRAS*|
|**Abstract**| In this paper we introduce the software package ProPane, written for the R data analysis language. ProPane combines the full range of wcslib projections with the C++ image manipulation routines provided by the CImg library. ProPane offers routines for image warping and combining (including stacking), and various related tasks such as image alignment tweaking and pixel masking. It can stack an effectively unlimited number of target frames using multiple parallel cores, and offers threading for many lower level routines. It has been used for a number of current and upcoming large surveys, and we present a range of its capabilities and features. ProPane is already available under a permissive open-source LGPL-3 license at github.com/asgr/ProPane (DOI: 10.5281/zenodo.10057053). |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2311.01982-b31b1b.svg)](https://arxiv.org/abs/arXiv:2311.01982) | **Optical monitoring of the Didymos-Dimorphos asteroid system with the  Danish telescope around the DART mission impact**  |
|| A. Rożek, et al. |
|*Appeared on*| *2023-11-06*|
|*Comments*| *20 pages, 6 figures. Accepted for publication in The Planetary Science Journal*|
|**Abstract**| The NASA's Double-Asteroid Redirection Test (DART) was a unique planetary defence and technology test mission, the first of its kind. The main spacecraft of the DART mission impacted the target asteroid Dimorphos, a small moon orbiting asteroid (65803) Didymos, on 2022 September 26. The impact brought up a mass of ejecta which, together with the direct momentum transfer from the collision, caused an orbital period change of 33 +/- 1 minutes, as measured by ground-based observations. We report here the outcome of the optical monitoring campaign of the Didymos system from the Danish 1.54 m telescope at La Silla around the time of impact. The observations contributed to the determination of the changes in the orbital parameters of the Didymos-Dimorphos system, as reported by arXiv:2303.02077, but in this paper we focus on the ejecta produced by the DART impact. We present photometric measurements from which we remove the contribution from the Didymos-Dimorphos system using a H-G photometric model. Using two photometric apertures we determine the fading rate of the ejecta to be 0.115 +/- 0.003 mag/d (in a 2" aperture) and 0.086 +/- 0.003 mag/d (5") over the first week post-impact. After about 8 days post-impact we note the fading slows down to 0.057 +/- 0.003 mag/d (2" aperture) and 0.068 +/- 0.002 mag/d (5"). We include deep-stacked images of the system to illustrate the ejecta evolution during the first 18 days, noting the emergence of dust tails formed from ejecta pushed in the anti-solar direction, and measuring the extent of the particles ejected sunward to be at least 4000 km. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2311.01721.md
    + _build/html/tmp_2311.01721/./nh2cho_map.ps
    + _build/html/tmp_2311.01721/./86036.ps
    + _build/html/tmp_2311.01721/./94117.ps
    + _build/html/tmp_2311.01721/./95310.ps
    + _build/html/tmp_2311.01721/./95565.ps
    + _build/html/tmp_2311.01721/./97481.ps
    + _build/html/tmp_2311.01721/./99819.ps
    + _build/html/tmp_2311.01721/./106434.ps
    + _build/html/tmp_2311.01721/./107207.ps
    + _build/html/tmp_2311.01721/./108243.ps
    + _build/html/tmp_2311.01721/./108734.ps
    + _build/html/tmp_2311.01721/./93247.ps
    + _build/html/tmp_2311.01721/./93810.ps
    + _build/html/tmp_2311.01721/./94272.ps
    + _build/html/tmp_2311.01721/./94636.ps
    + _build/html/tmp_2311.01721/./100344.ps
    + _build/html/tmp_2311.01721/./101226.ps
    + _build/html/tmp_2311.01721/./102567.ps
    + _build/html/tmp_2311.01721/./103170.ps
    + _build/html/tmp_2311.01721/./103625.ps
    + _build/html/tmp_2311.01721/./105396.ps


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\ncconh}{\mbox{NCCONH_2}}$</div>



<div id="title">

# Tentative detection of cyanoformamide $NCCONH_2$ in space

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2311.01721-b31b1b.svg)](https://arxiv.org/abs/2311.01721)<mark>Appeared on: 2023-11-06</mark> -  _20 pages, 6 figures, 2 tables, accepted by PASJ. arXiv admin note: text overlap with arXiv:2108.05001_

</div>
<div id="authors">

J. \. \altaffilmark{1,2}, et al.

</div>
<div id="abstract">

**Abstract:** The peptide-like molecules, cyanoformamide ( $\ncconh$ ), is the cyano (CN) derivative of formamide ($NH_2$ CHO). It is known to play a role in the synthesis of nucleic acid precursors under prebiotic conditions. In this paper, we present a tentative detection of $\ncconh$ in the interstellar medium (ISM) with the Atacama Large Millimeter/submillimeter Array (ALMA) archive data. Ten unblended lines of $\ncconh$ were seen around 3 $\sigma$ noise levels toward Sagittarius B2(N1E), a position that is slightly offset from the continuum peak. The column density of $\ncconh$ was estimated to be 2.4 $\times 10^{15}$ cm $^{-2}$ , and the fractional abundance of $\ncconh$ toward Sgr B2(N1E) was $6.9\times10^{-10}$ . The abundance ratio between $NCCONH_2$ and $NH_2$ CHO is estimated to be $\sim$ 0.01. We also searched for other peptide-like molecules toward Sgr B2(N1E). The abundances of $NH_2$ CHO, $CH_3$ NCO and $CH_3$ NHCHO toward Sgr B2(N1E) were about one tenth of those toward Sgr B2(N1S), while the abundances of $CH_3$ $CONH_2$ was only one twentieth of that toward Sgr B2(N1S).

</div>

<div id="div_fig1">

<img src="tmp_2311.01721/./nh2cho_map.ps" alt="Fig1" width="100%"/>

**Figure 1. -** Integrated intensity map of $NH_2$CHO and $CH_3^{13}$CN in Sgr B2(N1). The integrated intensity map of $NH_2$CHO at 103.525 GHz are shown in color scale. The integrated intensity map of $CH_3^{13}$CN at 91.94 GHz is shown in contours, which represent 30\% to 90\% percent of peak integrated intensity. The white crosses indicate the position of Sgr B2(N1E), Sgr B2(N1S), and the center of Sgr B2(N1). The white ellipse shows the size of the respective synthetic beam.
 (*f:sgrb2*)

</div>
<div id="div_fig2">

<img src="tmp_2311.01721/./86036.ps" alt="Fig2.1" width="10%"/><img src="tmp_2311.01721/./94117.ps" alt="Fig2.2" width="10%"/><img src="tmp_2311.01721/./95310.ps" alt="Fig2.3" width="10%"/><img src="tmp_2311.01721/./95565.ps" alt="Fig2.4" width="10%"/><img src="tmp_2311.01721/./97481.ps" alt="Fig2.5" width="10%"/><img src="tmp_2311.01721/./99819.ps" alt="Fig2.6" width="10%"/><img src="tmp_2311.01721/./106434.ps" alt="Fig2.7" width="10%"/><img src="tmp_2311.01721/./107207.ps" alt="Fig2.8" width="10%"/><img src="tmp_2311.01721/./108243.ps" alt="Fig2.9" width="10%"/><img src="tmp_2311.01721/./108734.ps" alt="Fig2.10" width="10%"/>

**Figure 2. -** Unblended transitions of $NCCONH_2$ toward Sgr B2(N1E). Black lines show continuum-subtracted spectrum observed with the ALMA telescope. The black dashed lines show the 3$\sigma$ noise levels. The median values of the rms noise levels in the channel maps were used here. The red lines show the modeling results. The cyan lines show the modeling results of other molecules. (*f:clean*)

</div>
<div id="div_fig3">

<img src="tmp_2311.01721/./93247.ps" alt="Fig3.1" width="10%"/><img src="tmp_2311.01721/./93810.ps" alt="Fig3.2" width="10%"/><img src="tmp_2311.01721/./94272.ps" alt="Fig3.3" width="10%"/><img src="tmp_2311.01721/./94636.ps" alt="Fig3.4" width="10%"/><img src="tmp_2311.01721/./100344.ps" alt="Fig3.5" width="10%"/><img src="tmp_2311.01721/./101226.ps" alt="Fig3.6" width="10%"/><img src="tmp_2311.01721/./102567.ps" alt="Fig3.7" width="10%"/><img src="tmp_2311.01721/./103170.ps" alt="Fig3.8" width="10%"/><img src="tmp_2311.01721/./103625.ps" alt="Fig3.9" width="10%"/><img src="tmp_2311.01721/./105396.ps" alt="Fig3.10" width="10%"/>

**Figure 3. -** Partially blended transitions of $NCCONH_2$ toward Sgr B2(N1E). Black lines show continuum-subtracted spectrum observed with the ALMA telescope. The black dashed lines show the 3$\sigma$ noise levels derived by \citet{Belloche19}. The red lines show the modeling results. The cyan lines show the modeling results of other molecules. (*f:partial*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2311.01721"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

316  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
